In [15]:
import numpy as np
import cv2
import math
import sys
sys.setrecursionlimit(15000)
doLogging = False
def showImage(img, name):
    cv2.imshow(name, img)
    return
def saveImage(img, name):
    cv2.imwrite(name + ".png", img)
    return
def GaussArea(pts):
    n = len(pts)
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += pts[i][0] * pts[j][1]
        area -= pts[j][0] * pts[i][1]
    area = abs(area)*0.5
    return area
def onePassDCE(ctrIn):
    length = ctrIn.shape[0]
    ctr = ctrIn
    K = np.zeros((length,1))
    for i in range(length):
        if i == length-1:
            theta = abs(np.arctan2((ctr[i,1] - ctr[i-1,1]),(ctr[i,0] - ctr[i-1,0])) - np.arctan2((ctr[0,1] - ctr[i,1]),(ctr[0,0] - ctr[i,0])))
            l = np.sqrt((ctr[i,1] - ctr[i-1,1])**2 + (ctr[i,0] - ctr[i-1,0])**2)
            l1 = np.sqrt((ctr[0,1] - ctr[i,1])**2 + (ctr[0,0] - ctr[i,0])**2)
        else:
            theta = abs(np.arctan2((ctr[i,1] - ctr[i-1,1]),(ctr[i,0] - ctr[i-1,0])) - np.arctan2((ctr[i+1,1] - ctr[i,1]),(ctr[i+1,0] - ctr[i,0])))
            l = np.sqrt((ctr[i,1] - ctr[i-1,1])**2 + (ctr[i,0] - ctr[i-1,0])**2)
            l1 = np.sqrt((ctr[i+1,1] - ctr[i,1])**2 + (ctr[i+1,0] - ctr[i,0])**2)
        K[i] = (theta*l*l1)/(l+l1)
    idx = K.argmin()
    trimmedContour = np.delete(ctrIn,idx,0)
    return trimmedContour

def Pavlidis(img, start, flag, points):

    origin = start
    c = origin[0]
    r = origin[1]
    point = origin
    count = 0
    while count < 3:
        if flag == "up":
            if img[r-1,c-1] > 0:
                flag = 'left'
                point = [c-1,r-1]
                points = np.append(points,point)
                break
            elif img[r-1,c] > 0:
                flag = 'up'
                point = [c,r-1]
                points = np.append(points, point)
                break
            elif img[r-1,c+1] > 0:
                flag = 'up'
                point = [c+1,r-1]
                points = np.append(points,point)
                break
            else:
                flag = 'right'
                count += 1
        elif flag == "down":
            if img[r+1,c+1] > 0:
                flag = 'right'
                point = [c+1,r+1]
                points = np.append(points,point)
                break
            elif img[r+1,c] > 0:
                flag = 'down'
                point = [c,r+1]
                points = np.append(points,point)
                break
            elif img[r+1,c-1] > 0:
                flag = 'down'
                point = [c-1,r+1]
                points= np.append(points,point)
                break
            else:
                flag = 'left'
                count += 1
        elif flag == 'right':
            if img[r-1,c+1] > 0:
                flag = 'up'
                point = [c+1,r-1]
                points = np.append(points,point)
                break
            elif img[r,c+1] > 0:
                flag = 'right'
                point = [c+1,r]
                points = np.append(points,point)
                break
            elif img[r+1,c+1] > 0:
                flag = 'right'
                point = [c+1,r+1]
                points = np.append(points,point)
                break
            else:
                flag = 'down'
                count += 1
        elif flag == 'left':
            if img[r+1,c-1] > 0:
                flag = 'down'
                point = [c-1,r+1]
                points= np.append(points,point)
                break
            elif img[r,c-1] > 0:
                flag = 'left'
                point = [c-1,r]
                points= np.append(points,point)
                break
            elif img[r-1,c-1] > 0:
                flag = 'left'
                point = [c-1,r-1]
                points= np.append(points,point)
                break
            else:
                flag = 'up'
                count += 1        
    if flag == 'NULL' or count == 3 or [points[-2],points[-1]] == [points[0],points[1]]:
        # print(points)
        points1 = zip(points[0::2],points[1::2])
        # print(list(points1))
        p = np.asarray(list(points1))
        # print(p.shape)+
        return p    
    else:
        tmp = Pavlidis(img,point,flag, points)
    points = tmp
    return points
def showContour(ctr, img, name):
    contourImage = img
    length = ctr.shape[0]
    for count in range(length):
        contourImage[ctr[count,1], ctr[count,0]] = 255
        cv2.line(contourImage,(ctr[count,0], ctr[count,1]), \
            (ctr[(count+1)%length, 0], ctr[(count+1)%length, 1]),(255,255,255),1)
    showImage(contourImage, name)
    saveImage(contourImage, name)

inputImage = cv2.imread('VAoutline.png', cv2.IMREAD_GRAYSCALE)
showImage(inputImage, "Input image")
thresh = 70;
binary = cv2.threshold(inputImage, thresh, 255, cv2.THRESH_BINARY)[1]
(height, width) = binary.shape
y = np.uint8(height/2)
for x in range(width):
    if (binary[y, x] > 0):
        break
init_flag = 'up'
contour = Pavlidis(binary, [x,y], init_flag, [x,y])
# contour = contour[:-1,:]
showContour(contour, inputImage, "CONTOUR")
print(contour.shape, GaussArea(contour))
print("numloops shape	Area")
for step in range(6):
    numLoops = math.floor(contour.shape[0]/2)
    for idx in range(numLoops):
        contour = onePassDCE(contour)
    showContour(contour, np.zeros_like(inputImage), "STEP"+str(step))
    print(numLoops, contour.shape, GaussArea(contour))
cv2.waitKey(0)


(1613, 2) 68298.0
numloops shape	Area
806 (807, 2) 68298.0
403 (404, 2) 68308.5
202 (202, 2) 68360.5
101 (101, 2) 68124.5
50 (51, 2) 68494.5
25 (26, 2) 68766.5


-1